# NeuralWave
### [Valerio Orlandini](https://github.com/valeriorlandini/)
[GitHub](https://github.com/valeriorlandini/neuralwavetables) | [Generator website](https://valeriorlandini.github.io/neuralwavetables/)

_This code is MIT licensed_

If necessary, install the required dependencies

In [ ]:
!pip install torch numpy matplotlib

Import the libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn

Set the latent space dimension, i.e. the number of parameters you will be able to tweak to generate the wavetable. In the web interface example they are 8. Experiment with other values, keeping in mind that the wavetables of the dataset are 600 samples long

In [ ]:
LATENT_DIM = 8

Autoencoder implementation

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, wavetable_size, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = LATENT_DIM
        
        # Encoder layers
        self.encoder = nn.Sequential(
            nn.Linear(wavetable_size, self.latent_dim * 3),  
            nn.Tanh(),                     
            nn.Linear(latent_dim * 3, self.latent_dim),  
            nn.Tanh()                       
        )
        
        # Decoder layers
        self.decoder = nn.Sequential(
            nn.Linear(self.latent_dim, wavetable_size), 
            nn.Tanh()                    
        )

    def forward(self, x):
        # Encode the input
        encoded = self.encoder(x)
        # Decode the encoded representation
        decoded = self.decoder(encoded)
        return decoded

Instantiate the autoencoder, using GPU if CUDA is available

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder = Autoencoder(600, LATENT_DIM).to(device)

Load the dataset and split it between train and test, you can change the batch size if you wish

In [ ]:
wt_all = np.load('wavetables.npy', allow_pickle = True)
wt_train = wt_all[0:15000]
wt_test = wt_all[15000:]


BATCH_SIZE = 32
train_dataloader = torch.utils.data.DataLoader(wt_train, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(wt_test, batch_size=BATCH_SIZE, shuffle=False)

Train the network, you can adjust the number of epochs

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.001)

NUM_EPOCHS = 20

for epoch in range(NUM_EPOCHS):
    autoencoder.train()
    running_loss = 0.0
    for batch in train_dataloader:
        optimizer.zero_grad()
        outputs = autoencoder(batch)
        loss = criterion(outputs, batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    train_loss = running_loss / len(train_dataloader)
    
    autoencoder.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_dataloader:
            outputs = autoencoder(batch)
            loss = criterion(outputs, batch)
            val_loss += loss.item()
    
    val_loss /= len(val_dataloader)
    
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

Once you trained the network, you can test how it reconstructs the original wavetables

In [ ]:
n = 10
input_data = torch.tensor(wt_test[200:n+200]) 

with torch.no_grad():
    reconstructed = autoencoder(input_data)

plt.figure(figsize=(20, 4))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.plot(input_data.numpy()[i])
    plt.title("original")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(True)

    ax = plt.subplot(2, n, i + 1 + n)
    plt.plot(reconstructed.numpy()[i])
    plt.title("reconstructed")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(True)
plt.show()

Show the weights and the biases of the decoder layer

In [ ]:
linear_layer = autoencoder.decoder[0]

weights = linear_layer.weight
biases = linear_layer.bias
torch.set_printoptions(profile="full")
print("Weights:", weights)
print("Biases:", biases)